In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr evaluate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.0/97.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.9/429.9 kB 29.0 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade accelerate             #  It will accelerate the GPU
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.57.2
Uninstalling transformers-4.57.2:
  Successfully uninstalled transformers-4.57.2
Found existing installation: accelerate 1.12.0
Uninstalling accelerate-1.12.0:
  Successfully uninstalled accelerate-1.12.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 17.5 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline, set_seed,AutoTokenizer,AutoModelForSeq2SeqLM
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
import pandas as pd

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
# we define model and autotokenizer
model = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model) #Pre-processing and text representation

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [ ]:
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model).to(device)  #This model is good for Text-summarization seq2seqLm

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [ ]:
dataset = load_dataset("knkarthick/samsum")

README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/14731 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14731
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
})

In [ ]:
dataset['train']['dialogue'][1]

'Olivia: Who are you voting for in this election? \nOliver: Liberals as always.\nOlivia: Me too!!\nOliver: Great'

In [ ]:
dataset['train']['summary'][1]

'Olivia and Olivier are voting for liberals in this election. '

In [ ]:
#define function to do tokenizer
def convert_examples(example_batch):
  input_encoding = tokenizer(example_batch['dialogue'],max_length=1024,truncation=True)

  with tokenizer.as_target_tokenizer():
    target_encoding = tokenizer(example_batch['summary'],max_length=128,truncation=True) #Here it is the context manager because by using tokenizer,we are creating new tokens for targeting labels


  return {
      'input_ids':input_encoding['input_ids'],
      'attention_mask':input_encoding['attention_mask'],
      'labels':target_encoding['input_ids']
  }

In [ ]:
dataset_tokenized = dataset.map(convert_examples,batched = True) # .map will help to apply the above function to entire data set

Map:   0%|          | 0/14731 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

In [ ]:
dataset_tokenized["train"]

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14731
})

In [ ]:
dataset_tokenized['train'][1]

{'id': '13728867',
 'dialogue': 'Olivia: Who are you voting for in this election? \nOliver: Liberals as always.\nOlivia: Me too!!\nOliver: Great',
 'summary': 'Olivia and Olivier are voting for liberals in this election. ',
 'input_ids': [18038,
  151,
  2632,
  127,
  119,
  6228,
  118,
  115,
  136,
  2974,
  152,
  10463,
  151,
  35884,
  130,
  329,
  107,
  18038,
  151,
  2587,
  314,
  1242,
  10463,
  151,
  1509,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [18038, 111, 34296, 127, 6228, 118, 33195, 115, 136, 2974, 107, 1]}

In [ ]:
# Training
from transformers import DataCollatorForSeq2Seq # it is used to collect the data in batches in memory

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer,model=model_pegasus)

In [ ]:
from transformers import  TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='pegasus-samsum',num_train_epochs=1,warmup_steps = 500,
    per_device_train_batch_size=1,per_device_eval_batch_size=1,
    weight_decay=0.01,logging_steps=10,
    eval_strategy ='steps',
    eval_steps=500,
    save_steps=1e6,
    gradient_accumulation_steps=16
)

In [ ]:
trainer = Trainer(model = model_pegasus,
                  processing_class = tokenizer,
                  data_collator = seq2seq_data_collator,
                  args = training_args,
                  train_dataset = dataset_tokenized['test'],
                  eval_dataset = dataset_tokenized['validation']
                  )

In [ ]:
pip install wandb

In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kesana-class2024 (kesana-class2024-the-university-of-texas-at-dallas) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
import wandb

# Initialize a Weights & Biases run
# Replace 'your-entity' with your Weights & Biases username or team name
# Replace 'your-project-name' with the name you want for your project
wandb.init(
    project="summarizati", # Example project name
    config={
        "learning_rate": 0.0001,
        "epochs": 1,
        "batch_size": 1,
        "model_name": "google/pegasus-cnn_dailymail"
    }
)

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Currently logged in as: kesana-class2024 (kesana-class2024-the-university-of-texas-at-dallas) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
#wanbd key-869bf53044af5b9d409305946bf0be5254245e34
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.


Step,Training Loss,Validation Loss


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=52, training_loss=2.978356067950909, metrics={'train_runtime': 635.661, 'train_samples_per_second': 1.288, 'train_steps_per_second': 0.082, 'total_flos': 314017624350720.0, 'train_loss': 2.978356067950909, 'epoch': 1.0})

In [ ]:
#Evaluation

# This wil create the batches from a list to process data in chunks rather than once
def generated_batch_sized_chunks(list_of_elements,batch_size):
  for i in range(0,len(list_of_elements) , batch_size):
    yield list_of_elements[i:i+batch_size]

# This is the main evaluation function
def calculate_metric_on_test_ds(dataset,metric,model,tokenizer,
                                batch_size=16,device=device,
                                column_text="article",column_summary="highlights"):
  # prepare the batches
  article_batches = list(generated_batch_sized_chunks(dataset[column_text],batch_size))
  target_batches = list(generated_batch_sized_chunks(dataset[column_summary],batch_size))

# zip(): Pairs up corresponding article and target batches
# tqdm(): Shows a progress bar during iteration

  for article_batch,target_batch in tqdm(
      zip(article_batches,target_batches),total=len(article_batches)):

# tokenizer(): Converts text to numerical tokens the model understands
# max_length=1024: Maximum input sequence length
# truncation=True: Cuts off text longer than 1024 tokens
# padding="max_length": Pads shorter sequences to 1024 tokens (ensures uniform batch size)
# return_tensors="pt": Returns PyTorch tensors

      inputs = tokenizer(article_batch,max_length=1024,
                         truncation = True,padding="max_length",return_tensors="pt")

# input_ids.to(device): Moves input tokens to GPU/CPU
# attention_mask.to(device): Mask indicating which tokens are real vs padding
# length_penalty=0.8: Slightly favors shorter summaries (< 1.0 = shorter, > 1.0 = longer)
# num_beams=8: Uses beam search with 8 beams (explores 8 possible sequences simultaneously for better quality)
# max_length=128: Maximum summary length in tokens

      summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                 attention_mask=inputs["attention_mask"].to(device),
                                 length_penalty=0.8,num_beams=8,max_length=128)

# Converts token IDs back to human-readable text
# skip_special_tokens=True: Removes special tokens like [CLS], [SEP], <pad>
# clean_up_tokenization_spaces=True: Fixes spacing issues (e.g., "don 't" → "don't")

      decoded_summaries = [tokenizer.decode(s,skip_special_tokens=True,
                                            clean_up_tokenization_spaces=True)
                           for s in summaries]

      # cleaning summaries
      decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

# metric calculator
      metric.add_batch(predictions=decoded_summaries,references=target_batch)
      # predictions - model-generated summaries
      # references - ground summaries from dataset

  score = metric.compute() # final metric across all batches
  return score

rouge1: Measures unigram (single word) overlap between generated and reference summaries

Example: "the cat sat" vs "the dog sat" → 2/3 words match



rouge2: Measures bigram (two consecutive words) overlap

Example: "the cat sat" vs "the dog sat" → "the cat" doesn't match "the dog", but if we had "cat sat" in both, it would count



rougeL: Measures Longest Common Subsequence (LCS)

Finds the longest sequence of words that appear in the same order (but not necessarily consecutively)
More flexible than n-gram matching



rougeLsum: Similar to rougeL but designed specifically for multi-sentence summaries

Splits text into sentences first, then computes LCS
Better for evaluating longer summaries with multiple sentences

In [ ]:
import evaluate

rouge_names = ["rouge1","rouge2","rougeL","rougeLsum"]
rouge_metric = evaluate.load('rouge')
rouge_metric

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value('string'), 'references': List(Value('string'))}, {'predictions': Value('string'), 'references': Value('string')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
 

In [ ]:
score = calculate_metric_on_test_ds(
    dataset['test'][0:10],rouge_metric,trainer.model,tokenizer,batch_size=2,column_text='dialogue',column_summary='summary'
)

rouge_dict = dict((rn,score[rn]) for rn in rouge_names)
pd.DataFrame(rouge_dict,index=[f'pegasus'])

100%|██████████| 5/5 [00:24<00:00,  4.98s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.023258,0.0,0.022668,0.02324


In [ ]:
##save model
model_pegasus.save_pretrained("pegasus-samsum-model")

In [29]:
## save tokenizer
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [32]:
#Load the tokenizer
tokernier_l = AutoTokenizer.from_pretrained("/content/tokenizer")

In [36]:
#Prediction

gen_kwargs = {"length_penalty":0.8,"num_beams":8,"max_length":128}

sample_text = dataset['test'][0]["dialogue"]

reference = dataset['test'][0]["summary"]

pipe = pipeline("summarization",model = "pegasus-samsum-model",tokenizer = tokernier_l)


print("Dialogue:")
print(sample_text)

print("\nReference Summary:")
print(reference)

print("\nModel Summary:")
print(pipe(sample_text,**gen_kwargs)[0]["summary_text"])

Device set to use cuda:0
Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary:
Amanda: Ask Larry Amanda: He called her last time we were at the park together .<n>Hannah: I'd rather you texted him .<n>Amanda: Just text him .
